In [1]:
from ortools.sat.python import cp_model
import numpy as np
import time

In [2]:
def convert_string_int(s):
    rs = s.split(" ")
    rs = [int(x) for x in rs]
    return rs


In [3]:
class Solver():
    def __init__(self,file_path): 
        with open(file=file_path) as f:
            line1 = f.readline()
            self.N, self.D = convert_string_int(line1)
            self.w = np.ones((self.N+1,self.N+1,self.D+1))*99
            line2 = f.readline()
            self.s, self.t = convert_string_int(line2)
            lines = f.readlines()
            for line in lines:
                s,t,w,c = convert_string_int(line)
                self.w[s][t][c] = min(self.w[s][t][c],w)
        f.close()
        self.model = cp_model.CpModel()
    def solve(self):
        timestart = time.time()
        x= {}
        for i in range(1, self.N+1):
            for j in range(1, self.N+1):
                for k in range(1,self.D+1):
                    x[str(i) + "," + str(j) + "," + str(k)] = self.model.NewIntVar(0, 1, "x[" + str(i) + "," + str(j)+ "," + str(k) + "]")

        t = {}
        for i in range(1,self.N+1):
            for j in range(1,self.D+1):
                t[str(i) + "," + str(j)] = self.model.NewIntVar(0, 1, "t[" + str(i) + "," + str(j) + "]")

        p = {}

        for i in range(1,self.D+1):
            p[str(i)] = self.model.NewIntVar(0,self.D, "p[" + str(i) + "]")
        
        a = {}

        for i in range(1,self.N+1):
            for k in range(1,self.D+1):
                a[str(i) + ","+str(k)] = self.model.NewIntVar(0,1, "a[" + str(i) + "," + str(k) +"]")
        b = {}

        for i in range(1,self.N+1):
            for k in range(1,self.D+1):
                b[str(i) + ","+str(k)] = self.model.NewIntVar(0,1, "b[" + str(i) + "," + str(k) +"]")

        c = {}

        for i in range(1,self.N+1):
            for k in range(1,self.D+1):
                c[str(i) + ","+str(k)] = self.model.NewIntVar(0,1, "c[" + str(i) + "," + str(k) +"]")

        for i in range(1,self.N+1):
            in_deg = x[str(1) + "," + str(i) + "," + str(1)]*1
            out_deg = x[str(i) + "," + str(1) + "," + str(1)]*1
            for j in range(1,self.N+1):
                for k in range(1,self.D+1):
                    if j == 1 and k == 1: 
                        continue
                    in_deg += x[str(j) + "," + str(i)+ "," + str(k)]
                    out_deg += x[str(i) + "," + str(j)+"," + str(k)]
            if i == self.s: 
                self.model.Add(in_deg == 0)
                self.model.Add(out_deg == 1)
            elif i == self.t:
                self.model.Add(in_deg == 1)
                self.model.Add(out_deg == 0)
            else:
                self.model.Add(in_deg == out_deg)
                self.model.Add(in_deg <=1)
                self.model.Add(out_deg <= 1)
            for k in range(1,self.D+1):
                exp = x[str(1) + ","+str(1)+","+str(k)]*1
                for i in range(1,self.N+1):
                    for j in range(1,self.N+1):  
                        exp += x[str(i) + ","+str(j)+","+str(k)]
                self.model.Add(p[str(k)] == exp)
            
            for i in range(1,self.N+1):
                for k in range(1,self.D+1):
                    exp_1 = x[str(i) + ","+str(1)+","+str(k)]*1
                    exp_2 = x[str(1) + ","+str(i)+","+str(k)]*1
                    for j in range(2,self.N+1):
                        exp_1  += x[str(i) + ","+str(j)+","+str(k)]
                        exp_2  += x[str(j) + ","+str(i)+","+str(k)]
                    self.model.Add(a[str(i) + ","+str(k)] == exp_1)
                    self.model.Add(b[str(i) + ","+str(k)] == exp_2)
                    self.model.AddMultiplicationEquality(c[str(i) + ","+str(k)],[a[str(i) + ","+str(k)],b[str(i) + ","+str(k)]])
            for i in range(1,self.N+1):
                for k in range(1,self.D+1):
                    stp = a[str(i) + ","+str(k)]*1
                    stp +=b[str(i) + ","+str(k)]*1
                    stp -=c[str(i) + ","+str(k)]*1
                    self.model.Add( t[str(i)+","+str(k)] == stp)
            # for i in range(1,self.N+1):
            #     for k in range(1,self.D+1):
            #         exp = x[str(i) + ","+str(1)+","+str(k)]*1
            #         exp+=x[str(1) + ","+str(i)+","+str(k)]*1
            #         a = self.model.NewIntVar(0,1,"a")
            #         self.model.AddMultiplicationEquality(a,[x[str(i) + ","+str(1)+","+str(k)],x[str(1) + ","+str(i)+","+str(k)]])
            #         exp-=a*1
            #         for j in range(2,self.N+1):
            #             exp += x[str(i) + ","+str(j)+","+str(k)]
            #             exp += x[str(j) + ","+str(i)+","+str(k)]
            #             b = self.model.NewIntVar(0,1,"b")
            #             self.model.AddMultiplicationEquality(a,[x[str(i) + ","+str(j)+","+str(k)],x[str(j) + ","+str(i)+","+str(k)]])
            #             exp-= b*1
            #         self.model.Add(exp == t[str(i)+","+str(k)])
            for k in range(1,self.D+1):
                exp = t[str(1)+","+str(k)]*1
                for i in range(2,self.N+1):
                    exp+= t[str(i)+","+str(k)]
                self.model.Add(exp - p[str(k)] <= 1)
                for k in range(1,self.D+1):
                    exp = t[str(1)+","+str(k)]*1
                    for i in range(2,self.N+1):
                        exp+= t[str(i)+","+str(k)]
                    self.model.Add(exp - p[str(k)] <= 1)
            func = self.model.NewIntVar(0,1000,"func")
            for i in range(1,self.N+1):
                for j in range(1,self.N+1):
                    for k in range(1,self.D+1):
                        func+= x[str(i) + ","+str(j)+","+str(k)] * self.w[i][j][k]
        self.model.Minimize(func)
        solver = cp_model.CpSolver()
        status = solver.Solve(self.model)
        if status == cp_model.OPTIMAL:
            print(solver.ObjectiveValue())
        print('Statistics')
        print(f'  status   : {solver.StatusName(status)}')
        print(f'  conflicts: {solver.NumConflicts()}')
        print(f'  branches : {solver.NumBranches()}')
        print(f'  wall time: {solver.WallTime()} s')
        print(f'  Run time: {time.time()-timestart} s')

In [4]:
fpath =  'MFEA_lib/tasks/Benchmark/__references__/IDPC_DU/IDPC_EDU/data/set1/idpc_10x20x2713.idpc'
sol = Solver(file_path =fpath)
sol.solve()

7.0
Statistics
  status   : OPTIMAL
  conflicts: 123
  branches : 7125
  wall time: 1.654487719 s
  Run time: 5.854344844818115 s


In [5]:
import os
path = "MFEA_lib/tasks/Benchmark/__references__/IDPC_DU/IDPC_EDU/data/set1"
dir_list = os.listdir(path)[:5]
for dir in dir_list:
    fpath = path+ '/' + dir
    print('\n'+dir)
    sol = Solver(fpath)
    sol.solve()



idpc_10x10x1000.idpc
7.0
Statistics
  status   : OPTIMAL
  conflicts: 0
  branches : 2304
  wall time: 0.506707401 s
  Run time: 2.0439984798431396 s

idpc_10x20x2713.idpc
7.0
Statistics
  status   : OPTIMAL
  conflicts: 161
  branches : 5646
  wall time: 1.921705285 s
  Run time: 5.867158889770508 s

idpc_10x5x425.idpc
7.0
Statistics
  status   : OPTIMAL
  conflicts: 1
  branches : 1153
  wall time: 0.24515123400000002 s
  Run time: 1.0940592288970947 s

idpc_15x15x3375.idpc
10.0
Statistics
  status   : OPTIMAL
  conflicts: 0
  branches : 7546
  wall time: 9.928559603 s
  Run time: 18.277137279510498 s

idpc_15x30x12111.idpc
10.0
Statistics
  status   : OPTIMAL
  conflicts: 0
  branches : 15552
  wall time: 31.887476992000003 s
  Run time: 48.34701919555664 s
